In [1]:
import sys
print(sys.executable)

e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Scripts\python.exe


In [2]:
import os
from dotenv import load_dotenv
import mlflow   
import numpy as np

load_dotenv()

e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


True

In [3]:
mlflow_tracking_uri=os.getenv('MLFLOW_TRACKING_URI')
mlflow_tracking_username=os.getenv('MLFLOW_TRACKING_USERNAME')
mlflow_tracking_password=os.getenv('MLFLOW_TRACKING_PASSWORD')

print(mlflow_tracking_uri)
print(mlflow_tracking_username)

https://dagshub.com/vipul100699/YouTube-Comments-Sentiment-Analyzer.mlflow
vipul100699


In [4]:
print("Tracking URI: ", mlflow.get_tracking_uri())

Tracking URI:  https://dagshub.com/vipul100699/YouTube-Comments-Sentiment-Analyzer.mlflow


In [5]:
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment("youtube-comment-analyzer-exp")

<Experiment: artifact_location='mlflow-artifacts:/5e01e4fa9917407d83a3c699fb4e1511', creation_time=1766646329030, experiment_id='1', last_update_time=1766646329030, lifecycle_stage='active', name='youtube-comment-analyzer-exp', tags={}>

In [6]:
with mlflow.start_run():
    mlflow.log_param("model", "baseline")
    mlflow.log_metric("metric1", np.random.rand())

## Creating Baseline Model

In [7]:
import numpy as np
import pandas as pd

In [8]:
# Read the dataset from the specified URL into a pandas DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
# Print the shape (number of rows and columns) of the DataFrame
print(df.shape)
# Display the first 5 rows of the DataFrame to inspect the data
print(df.head())

(37249, 2)
                                       clean_comment  category
0   family mormon have never tried explain them t...         1
1  buddhism has very much lot compatible with chr...         1
2  seriously don say thing first all they won get...        -1
3  what you have learned yours and only yours wha...         0
4  for your own benefit you may want read living ...         1


In [9]:
df.dropna(inplace=True)

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df = df[~(df['clean_comment'].str.strip() == '')]

In [12]:
print(df.shape)
df.sample(5)

(36793, 2)


,clean_comment,category
23890,top sanghi betrayals,1
23692,how can verify stuff like these any reputed so...,0
29673,political correctness banned india henceforth,0
14800,prescient trump versus president trump,0
18527,govt should also get out the business temples ...,0


In [13]:
import nltk
print(nltk.__version__)
print(nltk.__file__)


3.9.2
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\nltk\__init__.py


In [14]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stopwords.words("english")[:10]

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an']

In [15]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Vipul
[nltk_data]     Goel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Vipul
[nltk_data]     Goel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
def preprocess_comment_function(comment):
    comment = comment.lower()

    comment = comment.strip()
    
    comment = re.sub(r'\n', ' ', comment)

    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

In [19]:
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment_function)

In [20]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [21]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [22]:
# Step 1: Vectorize the comments using Bag of Words (CountVectorizer)
vectorizer = CountVectorizer(max_features=10000) # Bag of Words model with a limit of 1000 features

In [23]:
X = vectorizer.fit_transform(df['clean_comment']).toarray()
y = df['category']  # Assuming 'sentiment' as the target variable (0 or 1 for binary classification)

In [24]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
X.shape

(36793, 10000)

In [26]:

y

0        1
1        1
2       -1
3        0
4        1
        ..
37244    0
37245    1
37246    0
37247    1
37248    0
Name: category, Length: 36793, dtype: int64

In [27]:
y.shape

(36793,)

In [28]:
# Step 2: Set up the MLFlow tracking server
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [29]:
# Set or Create an experiment
mlflow.set_experiment("RF Baseline")

2025/12/25 14:10:27 INFO mlflow.tracking.fluent: Experiment with name 'RF Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/84e6eec3735b4947b4921ce46af1c427', creation_time=1766652028142, experiment_id='2', last_update_time=1766652028142, lifecycle_stage='active', name='RF Baseline', tags={}>

In [31]:
import os

ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

In [32]:
# Step 1: Split the data into 80% train and 20% test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Define and train a random forest baseline model using a simple train-test split
with mlflow.start_run() as run:
    # Log a description for the run
    mlflow.set_tag("mlflow.runName", "RandomForest_Baseline_TrainTestSplit")
    mlflow.set_tag("experiment_type", "baseline")
    mlflow.set_tag("model_type", "RandomForestClassifier")

    # Add a description
    mlflow.set_tag("description", "Baseline RandomForest model for sentiment analysis using Bag of Words (BoW) with")

    # Log parameters for the vectorizer
    mlflow.log_param("vectorizer_type", "CountVectorizer")
    mlflow.log_param("vectorizer_max_features", vectorizer.max_features)

    # Log Random Forest parameters
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Log metrics for each class and accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):       # For precision, recall, f1-score etc
            for metric, value in metrics.items():
                if metric != "support":
                    mlflow.log_metric(f"{label}_{metric}", value)

    # Confusion Matrix plot
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")

    # Save and log the confusion matrix plot
    cm_path = os.path.join(ARTIFACT_DIR, "confusion_matrix.png")
    plt.savefig(cm_path)
    plt.close()

    mlflow.log_artifact(cm_path)

    # Log the Random Forest model
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Optionally log the dataset itself (if its small enough)
    data_path = os.path.join(ARTIFACT_DIR, "dataset.csv")
    df.to_csv(data_path, index=False)

    mlflow.log_artifact(data_path)

# Display final accuracy
print(f"Accuracy: {accuracy}")

2025/12/25 20:15:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Accuracy: 0.6483217828509308


In [33]:
print(classification_report)

<function classification_report at 0x000002A618C61300>


In [34]:
df.to_csv('reddit_preprocessing.csv', index=False)

In [35]:
pd.read_csv('reddit_preprocessing.csv').head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1
